In [1]:
# Importing the necessary libraries for tuning the indicators using TuneTA
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from pandas_ta import log_return, percent_return

from freqtrade.configuration import Configuration
from freqtrade.data.history import load_pair_history
from freqtrade.enums import CandleType
from tuneta.tune_ta import TuneTA

/home/ftuser/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os
from pathlib import Path

# Change directory
# Modify this cell to insure that the output shows the correct path.
# Define all paths relative to the project root shown in the cell output
project_root = "/workspaces/freqtrade"
i=0
try:
    os.chdir(project_root)
    assert Path('LICENSE').is_file()
except:
    while i<4 and (not Path('LICENSE').is_file()):
        os.chdir(Path(Path.cwd(), '../'))
        i+=1
    project_root = Path.cwd()
print(Path.cwd())

/workspaces/freqtrade


In [3]:
config = Configuration.from_files(["user_data/config.json"])

# Define some constants
config["timeframe"] = "5m"
# Name of the strategy class
config["strategy"] = "SampleStrategy"
# Location of the data
data_location = config["datadir"]
# Pair to analyze - Only use one pair here
pair = "BTC/USDT"

In [4]:
candles = load_pair_history(
    datadir=data_location,
    timeframe=config["timeframe"],
    pair=pair,
    data_format="feather",  # Make sure to update this to your data
    candle_type=CandleType.SPOT,
)
# Make date column the index
candles.set_index("date", inplace=True)
candles.head()

,open,high,low,close,volume
date,,,,,
2021-10-10 00:00:00+00:00,54949.72,54968.98,54829.33,54879.80,442.60302
2021-10-10 00:05:00+00:00,54879.81,54907.64,54806.95,54869.96,318.64279
2021-10-10 00:10:00+00:00,54869.96,54899.91,54810.17,54873.90,256.84050
2021-10-10 00:15:00+00:00,54873.89,55049.99,54840.49,55049.98,194.25011
2021-10-10 00:20:00+00:00,55049.98,55119.43,55028.92,55087.83,232.04342


In [5]:
candles["percent_return"] = percent_return(candles["close"], offset=-1)
candles["log_return"] = log_return(candles["close"], offset=-1)
candles = candles.dropna(axis='index')
candles.tail()

,open,high,low,close,volume,percent_return,log_return
date,,,,,,,
2024-07-20 09:10:00+00:00,66552.19,66575.00,66540.00,66575.00,37.56644,-0.000075,-0.000075
2024-07-20 09:15:00+00:00,66575.00,66575.00,66546.81,66570.01,21.43424,-0.000447,-0.000447
2024-07-20 09:20:00+00:00,66570.01,66574.00,66527.99,66540.27,19.59652,0.000680,0.000680
2024-07-20 09:25:00+00:00,66540.26,66585.50,66540.26,66585.50,17.11026,0.001547,0.001546
2024-07-20 09:30:00+00:00,66585.49,66688.53,66585.49,66688.53,57.73133,0.000278,0.000278


In [6]:
X_train, X_test, y_train, y_test = train_test_split(candles[['open', 'high', 'low', 'close', 'volume']], candles['log_return'], test_size=0.3, random_state=42)
tt = TuneTA(n_jobs=4, verbose=True)

In [7]:
tt.fit(
    X_train,
    y_train,
    indicators=['all'],
    trials=100,
    early_stop=100,
)

[I 2024-07-21 07:44:24,079] A new study created in memory with name: tta.BBANDS(X.close, timeperiod=trial.suggest_int('timeperiod', 3, 180), )
[I 2024-07-21 07:44:24,096] A new study created in memory with name: tta.DEMA(X.close, timeperiod=trial.suggest_int('timeperiod', 3, 180), )
[I 2024-07-21 07:44:24,116] A new study created in memory with name: tta.EMA(X.close, timeperiod=trial.suggest_int('timeperiod', 3, 180), )
[I 2024-07-21 07:44:24,134] A new study created in memory with name: tta.HT_TRENDLINE(X.close, )
[I 2024-07-21 07:44:24,159] A new study created in memory with name: tta.KAMA(X.close, timeperiod=trial.suggest_int('timeperiod', 3, 180), )
[I 2024-07-21 07:44:24,190] A new study created in memory with name: tta.MA(X.close, timeperiod=trial.suggest_int('timeperiod', 3, 180), )
[I 2024-07-21 07:44:24,222] A new study created in memory with name: tta.MIDPOINT(X.close, timeperiod=trial.suggest_int('timeperiod', 3, 180), )
[I 2024-07-21 07:44:24,257] Trial 0 finished with valu

KeyboardInterrupt: 

[I 2024-07-21 07:44:42,778] Trial 1 finished with value: 0.01916863394519758 and parameters: {'accelerationlong': 133, 'accelerationshort': 109}. Best is trial 0 with value: 0.01916863394519758.
[I 2024-07-21 07:44:42,978] Trial 12 finished with value: 0.014120879418707315 and parameters: {'timeperiod': 61}. Best is trial 6 with value: 0.07690414556710061.
[I 2024-07-21 07:44:42,998] Trial 13 finished with value: 0.01150772646058459 and parameters: {'timeperiod': 60}. Best is trial 3 with value: 0.02184802263411745.
[I 2024-07-21 07:44:43,531] Trial 15 finished with value: 0.008434170817205714 and parameters: {'timeperiod': 34}. Best is trial 11 with value: 0.02976520159415712.
[I 2024-07-21 07:44:43,541] Trial 15 finished with value: 0.008231062070721216 and parameters: {'timeperiod': 78}. Best is trial 6 with value: 0.03217609108533368.
[I 2024-07-21 07:44:43,650] Trial 7 finished with value: 0.028519564418709127 and parameters: {'acceleration': 157}. Best is trial 0 with value: 0.02

In [11]:
tt.report(target_corr=True, features_corr=True)


Indicator Correlation to Target:

                                                                                       Correlation
-----------------------------------------------------------------------------------  -------------
fta_QSTICK_period_50                                                                      0.655413
fta_BASP_period_42                                                                        0.207859
pta_pvol                                                                                  0.192586
tta_CDLSHORTLINE                                                                          0.117774
pta_pvi_length_70                                                                         0.117409
tta_TRIX_timeperiod_19                                                                    0.107179
tta_CMO_timeperiod_142                                                                    0.104613
pta_cmo_length_142                                                        